In [23]:
import yfinance as yf
import pandas as pd 
import ta
import os

pd.set_option('display.max_columns', None)

In [24]:
#Variables de entorno
start_date = '2014-03-01'
end_date = '2025-03-10'

In [25]:
tickers = ['NVDA', '^GSPC', '^IXIC', '^DJI', 'SOXX', 'SMH', 'XSD', 'QQQ', '^VIX']
data = yf.download(tickers, start=start_date, end=end_date)
data.columns = data.columns.map(lambda x: f"{x[0]}_{x[1]}")

[*********************100%***********************]  9 of 9 completed


In [26]:
data.head()

,Close_NVDA,Close_QQQ,Close_SMH,Close_SOXX,Close_XSD,Close_^DJI,Close_^GSPC,Close_^IXIC,Close_^VIX,High_NVDA,High_QQQ,High_SMH,High_SOXX,High_XSD,High_^DJI,High_^GSPC,High_^IXIC,High_^VIX,Low_NVDA,Low_QQQ,Low_SMH,Low_SOXX,Low_XSD,Low_^DJI,Low_^GSPC,Low_^IXIC,Low_^VIX,Open_NVDA,Open_QQQ,Open_SMH,Open_SOXX,Open_XSD,Open_^DJI,Open_^GSPC,Open_^IXIC,Open_^VIX,Volume_NVDA,Volume_QQQ,Volume_SMH,Volume_SOXX,Volume_XSD,Volume_^DJI,Volume_^GSPC,Volume_^IXIC,Volume_^VIX
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-03,0.433211,82.071487,19.172905,22.439188,31.348392,16168.030273,1845.729980,4277.299805,16.00,0.433211,82.291155,19.208272,22.483320,31.452411,16321.709961,1857.680054,4284.149902,16.780001,0.421842,81.339283,18.996075,22.245012,31.064694,16071.250000,1834.439941,4239.649902,15.38,0.425632,81.906740,19.141960,22.400941,31.206543,16321.709961,1857.680054,4261.419922,16.469999,375948000,38885700,3193000,269400,164000,92760000,3428220000,2077500000,0
2014-03-04,0.437711,83.114906,19.424887,22.836369,32.128551,16395.880859,1873.910034,4351.970215,14.10,0.440553,83.270506,19.464674,22.874617,32.204205,16419.490234,1876.229980,4357.209961,14.540000,0.434632,82.886091,19.340893,22.730453,31.783390,16169.320312,1849.229980,4327.540039,14.00,0.435343,82.968468,19.376260,22.730453,31.783390,16169.320312,1849.229980,4327.850098,14.530000,260344000,33189000,3212400,889800,48800,96120000,3765770000,2477850000,0
2014-03-05,0.441501,83.343719,19.446995,22.880501,32.218391,16360.179688,1873.810059,4357.970215,13.89,0.441738,83.490165,19.526567,22.942286,32.289316,16406.550781,1876.530029,4362.500000,14.320000,0.435343,83.078293,19.411628,22.798124,32.081271,16343.959961,1871.109985,4344.149902,13.81,0.437238,83.252195,19.486782,22.871676,32.166380,16395.880859,1874.050049,4352.759766,14.130000,241472000,27203300,3634000,289200,514000,73980000,3392990000,2215980000,0
2014-03-06,0.435343,83.270493,19.654770,23.039371,32.298771,16421.890625,1877.030029,4352.129883,14.21,0.444580,83.599980,19.659192,23.059967,32.383880,16450.169922,1881.939941,4371.709961,14.420000,0.434632,82.968455,19.504463,22.865791,32.156925,16360.559570,1874.180054,4341.000000,13.73,0.442212,83.535912,19.535409,22.948168,32.379154,16360.559570,1874.180054,4368.810059,13.820000,290144000,24063900,2289000,427800,94800,75900000,3360450000,2136260000,0
2014-03-07,0.434869,82.858604,19.654770,23.012892,32.213661,16452.720703,1878.040039,4336.220215,14.11,0.437948,83.618273,19.747606,23.189415,32.440619,16505.699219,1883.569946,4371.390137,14.430000,0.431553,82.437585,19.592879,22.942282,32.171109,16398.859375,1870.560059,4319.149902,13.51,0.437238,83.535896,19.716660,23.189415,32.435890,16424.529297,1878.520020,4370.979980,13.510000,268452000,31167800,2186800,672000,28400,80690000,3564740000,2175560000,0


## Crear indicadores tecnicos

In [27]:
data['rsi'] = ta.momentum.RSIIndicator(close=data['Close_NVDA'], window=14).rsi()

# Calcular MACD
macd = ta.trend.MACD(close=data['Close_NVDA'])
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()
data['macd_diff'] = macd.macd_diff()

# BBands
bb = ta.volatility.BollingerBands(close=data['Close_NVDA'], window=20, window_dev=2)
data['bb_mavg'] = bb.bollinger_mavg()    
data['bb_high'] = bb.bollinger_hband()     
data['bb_low'] = bb.bollinger_lband() 

stoch = ta.momentum.StochasticOscillator(
    high=data['High_NVDA'],
    low=data['Low_NVDA'],
    close=data['Close_NVDA'],
    window=14,
    smooth_window=3
)
data['stoch_k'] = stoch.stoch()
data['stoch_d'] = stoch.stoch_signal()

In [28]:
data

,Close_NVDA,Close_QQQ,Close_SMH,Close_SOXX,Close_XSD,Close_^DJI,Close_^GSPC,Close_^IXIC,Close_^VIX,High_NVDA,High_QQQ,High_SMH,High_SOXX,High_XSD,High_^DJI,High_^GSPC,High_^IXIC,High_^VIX,Low_NVDA,Low_QQQ,Low_SMH,Low_SOXX,Low_XSD,Low_^DJI,Low_^GSPC,Low_^IXIC,Low_^VIX,Open_NVDA,Open_QQQ,Open_SMH,Open_SOXX,Open_XSD,Open_^DJI,Open_^GSPC,Open_^IXIC,Open_^VIX,Volume_NVDA,Volume_QQQ,Volume_SMH,Volume_SOXX,Volume_XSD,Volume_^DJI,Volume_^GSPC,Volume_^IXIC,Volume_^VIX,rsi,macd,macd_signal,macd_diff,bb_mavg,bb_high,bb_low,stoch_k,stoch_d
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-03,0.433211,82.071487,19.172905,22.439188,31.348392,16168.030273,1845.729980,4277.299805,16.000000,0.433211,82.291155,19.208272,22.483320,31.452411,16321.709961,1857.680054,4284.149902,16.780001,0.421842,81.339283,18.996075,22.245012,31.064694,16071.250000,1834.439941,4239.649902,15.380000,0.425632,81.906740,19.141960,22.400941,31.206543,16321.709961,1857.680054,4261.419922,16.469999,375948000,38885700,3193000,269400,164000,92760000,3428220000,2077500000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-04,0.437711,83.114906,19.424887,22.836369,32.128551,16395.880859,1873.910034,4351.970215,14.100000,0.440553,83.270506,19.464674,22.874617,32.204205,16419.490234,1876.229980,4357.209961,14.540000,0.434632,82.886091,19.340893,22.730453,31.783390,16169.320312,1849.229980,4327.540039,14.000000,0.435343,82.968468,19.376260,22.730453,31.783390,16169.320312,1849.229980,4327.850098,14.530000,260344000,33189000,3212400,889800,48800,96120000,3765770000,2477850000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-05,0.441501,83.343719,19.446995,22.880501,32.218391,16360.179688,1873.810059,4357.970215,13.890000,0.441738,83.490165,19.526567,22.942286,32.289316,16406.550781,1876.530029,4362.500000,14.320000,0.435343,83.078293,19.411628,22.798124,32.081271,16343.959961,1871.109985,4344.149902,13.810000,0.437238,83.252195,19.486782,22.871676,32.166380,16395.880859,1874.050049,4352.759766,14.130000,241472000,27203300,3634000,289200,514000,73980000,3392990000,2215980000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-06,0.435343,83.270493,19.654770,23.039371,32.298771,16421.890625,1877.030029,4352.129883,14.210000,0.444580,83.599980,19.659192,23.059967,32.383880,16450.169922,1881.939941,4371.709961,14.420000,0.434632,82.968455,19.504463,22.865791,32.156925,16360.559570,1874.180054,4341.000000,13.730000,0.442212,83.535912,19.535409,22.948168,32.379154,16360.559570,1874.180054,4368.810059,13.820000,290144000,24063900,2289000,427800,94800,75900000,3360450000,2136260000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-07,0.434869,82.858604,19.654770,23.012892,32.213661,16452.720703,1878.040039,4336.220215,14.110000,0.437948,83.618273,19.747606,23.189415,32.440619,16505.699219,1883.569946,4371.390137,14.430000,0.431553,82.437585,19.592879,22.942282,32.171109,16398.859375,1870.560059,4319.149902,13.510000,0.437238,83.535896,19.716660,23.189415,32.435890,16424.529297,1878.520020,4370.979980,13.510000,268452000,31167800,2186800,672000,28400,80690000,3564740000,2175560000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,114.059998,497.049988,223.009995,201.149994,217.929993,43191.238281,5849.720215,18350.189453,22.780001,123.699997,513.039978,235.369995,211.649994,231.360001,44033.781250,5986.089844,18992.300781,24.309999,112.279999,493.579987,220.949997,199.320007,216.119995,42919.308594,5810.910156,18216.630859,19.250000,123.510002,511.519989,235.369995,211.509995,231.360001,43900.488281,5968.330078,18923.359375,19.830000,411381400,44587200,10001200,5067800,79800,731560000,5613850000,8170140000,0,36.555420,-2.694498,-1.030216,-1.664283,129.539999,144.670167,114.409831,5.712447,12.606277
2025-03-04,115.989998,495.549988,225.229996,202.270004,218.440002,42520.98

## Time related vars

In [29]:
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Dayofweek'] = data.index.day_of_week

## Guardar los datos

In [30]:
save_path = os.path.join('..', 'data', f'nvidia_data_{start_date}_{end_date}')
data.to_csv(save_path)

In [31]:
save_path

'../data/nvidia_data_2014-03-01_2025-03-10'

In [32]:
data

,Close_NVDA,Close_QQQ,Close_SMH,Close_SOXX,Close_XSD,Close_^DJI,Close_^GSPC,Close_^IXIC,Close_^VIX,High_NVDA,High_QQQ,High_SMH,High_SOXX,High_XSD,High_^DJI,High_^GSPC,High_^IXIC,High_^VIX,Low_NVDA,Low_QQQ,Low_SMH,Low_SOXX,Low_XSD,Low_^DJI,Low_^GSPC,Low_^IXIC,Low_^VIX,Open_NVDA,Open_QQQ,Open_SMH,Open_SOXX,Open_XSD,Open_^DJI,Open_^GSPC,Open_^IXIC,Open_^VIX,Volume_NVDA,Volume_QQQ,Volume_SMH,Volume_SOXX,Volume_XSD,Volume_^DJI,Volume_^GSPC,Volume_^IXIC,Volume_^VIX,rsi,macd,macd_signal,macd_diff,bb_mavg,bb_high,bb_low,stoch_k,stoch_d,Year,Month,Dayofweek
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-03-03,0.433211,82.071487,19.172905,22.439188,31.348392,16168.030273,1845.729980,4277.299805,16.000000,0.433211,82.291155,19.208272,22.483320,31.452411,16321.709961,1857.680054,4284.149902,16.780001,0.421842,81.339283,18.996075,22.245012,31.064694,16071.250000,1834.439941,4239.649902,15.380000,0.425632,81.906740,19.141960,22.400941,31.206543,16321.709961,1857.680054,4261.419922,16.469999,375948000,38885700,3193000,269400,164000,92760000,3428220000,2077500000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,3,0
2014-03-04,0.437711,83.114906,19.424887,22.836369,32.128551,16395.880859,1873.910034,4351.970215,14.100000,0.440553,83.270506,19.464674,22.874617,32.204205,16419.490234,1876.229980,4357.209961,14.540000,0.434632,82.886091,19.340893,22.730453,31.783390,16169.320312,1849.229980,4327.540039,14.000000,0.435343,82.968468,19.376260,22.730453,31.783390,16169.320312,1849.229980,4327.850098,14.530000,260344000,33189000,3212400,889800,48800,96120000,3765770000,2477850000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,3,1
2014-03-05,0.441501,83.343719,19.446995,22.880501,32.218391,16360.179688,1873.810059,4357.970215,13.890000,0.441738,83.490165,19.526567,22.942286,32.289316,16406.550781,1876.530029,4362.500000,14.320000,0.435343,83.078293,19.411628,22.798124,32.081271,16343.959961,1871.109985,4344.149902,13.810000,0.437238,83.252195,19.486782,22.871676,32.166380,16395.880859,1874.050049,4352.759766,14.130000,241472000,27203300,3634000,289200,514000,73980000,3392990000,2215980000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,3,2
2014-03-06,0.435343,83.270493,19.654770,23.039371,32.298771,16421.890625,1877.030029,4352.129883,14.210000,0.444580,83.599980,19.659192,23.059967,32.383880,16450.169922,1881.939941,4371.709961,14.420000,0.434632,82.968455,19.504463,22.865791,32.156925,16360.559570,1874.180054,4341.000000,13.730000,0.442212,83.535912,19.535409,22.948168,32.379154,16360.559570,1874.180054,4368.810059,13.820000,290144000,24063900,2289000,427800,94800,75900000,3360450000,2136260000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,3,3
2014-03-07,0.434869,82.858604,19.654770,23.012892,32.213661,16452.720703,1878.040039,4336.220215,14.110000,0.437948,83.618273,19.747606,23.189415,32.440619,16505.699219,1883.569946,4371.390137,14.430000,0.431553,82.437585,19.592879,22.942282,32.171109,16398.859375,1870.560059,4319.149902,13.510000,0.437238,83.535896,19.716660,23.189415,32.435890,16424.529297,1878.520020,4370.979980,13.510000,268452000,31167800,2186800,672000,28400,80690000,3564740000,2175560000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,114.059998,497.049988,223.009995,201.149994,217.929993,43191.238281,5849.720215,18350.189453,22.780001,123.699997,513.039978,235.369995,211.649994,231.360001,44033.781250,5986.089844,18992.300781,24.309999,112.279999,493.579987,220.949997,199.320007,216.119995,42919.308594,5810.910156,18216.630859,19.250000,123.510002,511.519989,235.369995,211.509995,231.360001,43900.488281,5968.330078,18923.359375,19.830000,411381400,44587200,10001200,5067800,79800,731560000,5613850000,8170140000,0,36.555420,-2.694498,-1.030216,-1.664283,129.539999,144.670167,114.409831,5.712447,12.